In [ ]:
from esper.prelude import *
from query.models import Shot
from django.db.models import Value
from django.db.models.fields import IntegerField
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.domain_interval_collection import DomainIntervalCollection

def topN(gen,n=1000):
    from tqdm import tqdm_notebook as tqdm
    from rekall.runtime import disjoint_domain_combiner
    result = None
    count = 0
    with tqdm(total=n) as pbar:
        for collection in gen:
            delta = sum(collection.size().values())
            pbar.update(delta)
            count += delta
            if result is None:
                result = collection
            else:
                result = disjoint_domain_combiner(result, collection)
            if count >= n:
                break
    return result       

def convert_to_1d_collection(collection):
    from rekall.interval_list import Interval
    video_map = collection.get_grouped_intervals()
    return VideoIntervalCollection({vid: [Interval(
        i.t[0], i.t[1], i.payload) for i in video_map[vid].get_intervals()] for vid in video_map})

def display_result(collection_1d, display_payload=False):
    from esper.rekall import intrvllists_to_result_bbox
    from esper.rekall import intrvllists_to_result_with_objects
    if display_payload:
        results = intrvllists_to_result_with_objects(collection_1d.get_allintervals(), 
            lambda p, v: p, limit=1000, stride=1)
    else:
        results = intrvllists_to_result_with_objects(collection_1d.get_allintervals(), 
            lambda p, v:[], limit=1000, stride=1)
    return esper_widget(results,
            crop_bboxes=False, show_middle_frame=False, disable_captions=True,
            results_per_page=25, jupyter_keybindings=True)

# Takes a function from arg -> IntervalSet3D, returns a function that runs on batches:
# [arg] -> DomainIntervalCollection where the domain key is arg.
def in_sequence(func):
    def fn(args):
        return DomainIntervalCollection({
            arg: func(arg) for arg in args
        })
    return fn

# Hermione in the Middle

In [ ]:
def hermione_in_the_middle(vids=None):
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d_utils import T,P,XY,or_preds,X
    from rekall.temporal_predicates import before, overlaps_before
    from rekall.bbox_predicates import height_at_least, same_value, same_height, left_of
    from query.models import FaceCharacterActor, Video
    
    MIN_FACE_HEIGHT = 0.25
    EPSILON = 0.15
    NAMES = [ 'ron weasley', 'hermione granger', 'harry potter' ]
    
    if vids is None:
        vids = [v.id for v in Video.objects.filter(name__contains='harry potter')]
    
    faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
        min_frame=F('face__frame__number'),
        max_frame=F('face__frame__number'),
        video_id=F('face__frame__video_id'),
        bbox_x1=F('face__bbox_x1'),
        bbox_y1=F('face__bbox_y1'),
        bbox_x2=F('face__bbox_x2'),
        bbox_y2=F('face__bbox_y2'),
        character_name=F('characteractor__character__name')
    ).filter(face__frame__video_id__in=vids)
    
    total = faces_with_character_actor_qs.count()
    
    schema = DomainIntervalCollection.django_bbox_default_schema()
    schema['payload'] = 'character_name'    
    all_faces = DomainIntervalCollection.from_django_qs(faces_with_character_actor_qs, schema, progress=True,
                                                    total=total)
    frames_with_faces = all_faces.group_by_time()
    
    def name_is(name):
        return lambda f: f.payload == name
    def start_before():
        return or_preds(overlaps_before(), before())
    def in_order():
        return lambda a,b,c: start_before()(a,b) and start_before()(b,c)
    def rev_order():
        return lambda *args: in_order()(*args[::-1])
    
    pattern = [
        (["harry"], [XY(height_at_least(MIN_FACE_HEIGHT)), name_is(NAMES[2])]),
        (["ron"], [XY(height_at_least(MIN_FACE_HEIGHT)), name_is(NAMES[0])]),
        (["hermione"], [XY(height_at_least(MIN_FACE_HEIGHT)), name_is(NAMES[1])]),
        (["harry", "ron"], [XY(same_value('y1', epsilon=EPSILON)), XY(same_height(epsilon=EPSILON))]),
        (["harry", "hermione"], [XY(same_value('y1', epsilon=EPSILON)), XY(same_height(epsilon=EPSILON))]),
        (["ron", "hermione"], [XY(same_value('y1', epsilon=EPSILON)), XY(same_height(epsilon=EPSILON))]),
        (["harry","hermione", "ron"], [X(or_preds(in_order(), rev_order()))])
    ]
    
    def matches_pattern(pattern, exact):
        def pred(intervals):
            return len(intervals.match(pattern, exact))>0
        return pred
    
    # Frame_IS<Face_IS<character>>
    final = frames_with_faces.filter(P(matches_pattern(pattern, exact=True)))
    return final

def payload_to_vgrid_objects(faces):
    from query.models import Character
    def intrvl_to_obj(face):
        return {
            'type': 'bbox',
            'bbox_x1': face.x[0], 'bbox_x2': face.x[1],
            'bbox_y1': face.y[0], 'bbox_y2': face.y[1],
            'character_id': Character.objects.get(name=face.payload).id
        }
    def update(acc, face):
        acc.append(intrvl_to_obj(face))
        return acc
    return faces.fold(update, [])
    
answer = hermione_in_the_middle()
print("Query Done. Preparing for VGrid.")
display_result(convert_to_1d_collection(answer.map_payload(payload_to_vgrid_objects)), display_payload=True)

# Kissing

In [ ]:
def kissing(vids):
    from query.models import Face
    from rekall.interval_set_3d import Interval3D, IntervalSet3D
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d_utils import T,P,XY,or_preds,X,Y,and_preds
    from rekall.merge_ops import payload_plus
    from rekall.payload_predicates import payload_satisfies
    from rekall.spatial_predicates import scene_graph
    from rekall.temporal_predicates import overlaps, overlaps_before, before
    from rekall.face_landmark_predicates import looking_left, looking_right
    from rekall.bbox_predicates import height_at_least, same_height
    import esper.face_landmarks_wrapper as flw
    from esper.captions import get_all_segments
    from tqdm import tqdm as tqdm
    
    MAX_MOUTH_DIFF = 0.12
    MIN_FACE_CONFIDENCE = 0.8
    MIN_FACE_HEIGHT = 0.4
    MAX_FACE_HEIGHT_DIFF = 0.1
    MIN_FACE_OVERLAP_X = 0.05
    MIN_FACE_OVERLAP_Y = 0.2
    MAX_FACE_OVERLAP_X_FRACTION = 0.7
    MIN_FACE_ANGLE = 0.2
    
    def get_landmarks(faces):
        ids = [face.payload for face in faces.get_intervals()]
        landmarks = flw.get_from_face_ids(ids)
        id_to_lm = {idx: lm for idx, lm in zip(ids, landmarks)}
        return faces.map_payload(lambda idx : {
            'id': idx,
            'landmarks': id_to_lm[idx]
        })
    
    def mouths_are_close(lm1, lm2):
        select_outer=[2,3,4,8,9,10]
        select_inner=[1,2,3,5,6,7]
        mouth1 = np.concatenate((lm1.outer_lips()[select_outer], lm1.inner_lips()[select_inner]))
        mouth2 = np.concatenate((lm2.outer_lips()[select_outer], lm2.inner_lips()[select_inner]))
        mean1 = np.mean(mouth1, axis=0)
        mean2 = np.mean(mouth2, axis=0)
        return np.linalg.norm(mean1-mean2) <= MAX_MOUTH_DIFF
    
    # Line is ax+by+c=0
    def project_point_to_line(pt, a, b, c):
        x0,y0=pt[0], pt[1]
        d=a*a+b*b
        x=(b*(b*x0-a*y0)-a*c)/d
        y=(a*(-b*x0+a*y0)-b*c)/d
        return np.array([x,y])
    
    # Returns (a,b,c) which defines ax+by+c=0
    def find_best_line_fit(xs, ys):
        fit1 = np.polyfit(xs, ys, 1)
        error1 = np.sum((np.poly1d(fit1)(xs)-ys)**2)
        fit2 = np.polyfit(ys, xs, 1)
        error2 = np.sum((np.poly1d(fit2)(ys)-xs)**2)
        if error1 < error2:
            # fit1[0]x+fit1[1]=y
            return fit1[0], -1, fit1[1]
        # fit2[0]y+fit2[1]=x
        return -1, fit2[0], fit2[1]
    
    # Positive if facing left
    def signed_face_angle(lm):
        center_line_indices = [27,28, 32, 33,34, 51,62,66,57]
        data = lm.landmarks[center_line_indices]
        a, b, c = find_best_line_fit(data[:,0], data[:,1])
        A = project_point_to_line(lm.landmarks[center_line_indices[0]], a, b, c)
        B = project_point_to_line(lm.landmarks[center_line_indices[-1]], a, b, c)
        AB = B-A
        AB = AB / np.linalg.norm(AB)
        C = np.mean(lm.nose_bridge()[2:4], axis=0)
        AC = C-A
        AC = AC / np.linalg.norm(AC)
        return np.cross(AB, AC)

    # Annotate face rows with start and end frames and the video ID
    faces_qs = Face.objects.filter(
        frame__regularly_sampled=True,
        probability__gte=MIN_FACE_CONFIDENCE).annotate(
        min_frame=F('frame__number'),
        max_frame=F('frame__number'),
        height = F('bbox_y2')-F('bbox_y1'),
        video_id=F('frame__video_id')).filter(height__gte=MIN_FACE_HEIGHT, video_id__in=vids)
    
    total = faces_qs.count()
    
    # Frame_IS<Face_IS<face_id>>
    frames_with_faces = DomainIntervalCollection.from_django_qs(
        faces_qs, DomainIntervalCollection.django_bbox_default_schema(),
        progress=True, total=total
    ).group_by_time(profile=True)
    
    overlap_faces_pattern = [
        (["left","right"], [
            X(or_preds(before(), overlaps_before())), # Left face on the left
            X(lambda f1,f2: f1.end - f2.start > MIN_FACE_OVERLAP_X), # Faces overlap
            Y(lambda f1,f2: min(f1.end, f2.end)-max(f1.start, f2.start) > MIN_FACE_OVERLAP_Y), # No face is entirely above another
            XY(same_height(MAX_FACE_HEIGHT_DIFF)),
            X(lambda f1, f2: (f1.end-f2.start)/max(f1.length(), f2.length()) < MAX_FACE_OVERLAP_X_FRACTION),
        ])
    ]
    
    def matches_pattern(pattern, exact):
        def pred(intervals):
            return len(intervals.match(pattern, exact))>0
        return pred
    
    # Frame_IS<Face_IS<face_id>>
    frames_with_overlapped_faces = frames_with_faces.filter(P(
        matches_pattern(overlap_faces_pattern, exact=True)), profile=True)
    
    # Frame_IS<Face_IS<FaceMeta>>
    frames_with_landmarks = frames_with_overlapped_faces.map_payload(get_landmarks, profile=True)
    opposing_face_pattern = [
        (['left'], [P(lambda f: signed_face_angle(f['landmarks']) < -MIN_FACE_ANGLE)]),
        (['right'], [P(lambda f: signed_face_angle(f['landmarks']) > MIN_FACE_ANGLE)]),
        (['left','right'], [P(lambda l, r: mouths_are_close(l['landmarks'], r['landmarks']))])
    ]
    
    # Frame_IS<Face_IS<FaceMeta>>
    frames_with_opposing_faces = frames_with_landmarks.filter(P(matches_pattern(opposing_face_pattern, exact=True)),
                                                              profile=True)
    
    vids = frames_with_opposing_faces.get_grouped_intervals().keys()
    
    # Merge with shots
    shots_qs = Shot.objects.filter(
        video_id__in = vids,
        cinematic = True,
    )
    total = shots_qs.count()
    # Shot_IS<>
    shots = DomainIntervalCollection.from_django_qs(
        shots_qs,
        progress=True, total=total
    )
    # Shot_IS<Frame_IS<Face_IS<FaceMeta>>>
    kissing_shots = shots.collect_by_interval(
        frames_with_opposing_faces,
        T(overlaps()),
        time_window=1, profile=True
    ).map_payload(lambda p:p[1]).map(
        # Take the start of the kissing as the start of the shot
        lambda shot: Interval3D((shot.payload.get_intervals()[0].t[0], shot.t[1]), payload=shot.payload))
    
    # Get faces in shots
    faces_qs2 = Face.objects.filter(
         frame__regularly_sampled=True,
         frame__video_id__in=vids,
         probability__gte=MIN_FACE_CONFIDENCE
    ).annotate(
             min_frame=F('frame__number'),
             max_frame=F('frame__number'),
             video_id=F('frame__video_id')       
    )
    total = faces_qs2.count()
    # Frame_IS<Face_IS>
    frames_with_faces2 = DomainIntervalCollection.from_django_qs(
        faces_qs2, DomainIntervalCollection.django_bbox_default_schema(),
        progress=True, total=total
    ).group_by_time(profile=True)
    
    
    
    def both_faces_are_high(faces):
        f1, f2 = faces.get_intervals()
        return f1.height() >= MIN_FACE_HEIGHT and f2.height() >= MIN_FACE_HEIGHT
        
    # Frame_IS<Face_IS>
    frames_with_two_faces = frames_with_faces2.filter(
        P(and_preds(lambda faces: faces.size()==2, both_faces_are_high)),
        profile=True
    )
    
    # Collect frames with two faces into kissing shots, and clips the shot to the last frame with two faces
    def clip_to_last_frame(intrvl):
        frames = intrvl.payload[1]
        if frames.empty():
            return intrvl.copy()
        return Interval3D((intrvl.t[0], frames.get_intervals()[-1].t[1]), payload=intrvl.payload)
    
    # Shot_IS<(Frame_IS<Face_IS>, Frame_IS<Face_IS>)>
    clipped_kissing_shots = kissing_shots.collect_by_interval(
        frames_with_two_faces, T(overlaps()), time_window=1, filter_empty=False, profile=True
    ).map(clip_to_last_frame).filter_size(min_size=12)
    
    
    results = get_all_segments(vids)
    fps_map = dict((i, Video.objects.get(id=i).fps) for i in vids)
    # Word_IS<>
    caption_results = DomainIntervalCollection({
        video_id: IntervalSet3D([Interval3D((
            word[0] * fps_map[video_id], # start frame
            word[1] * fps_map[video_id]))
            for word in words])
        for video_id, words in tqdm(results)
    })
    
    kissing_without_words = clipped_kissing_shots.minus(caption_results, profile=True)
    kissing_final = kissing_without_words.temporal_coalesce(epsilon=0.5, profile=True).map(
        lambda i: Interval3D((int(i.t[0]), int(i.t[1])), payload=i.payload)
    ).filter_size(min_size=12)
    
    return kissing_final

def payload_to_vgrid_objects(payload):
    # Frame_IS<Face_IS<FaceMeta>>
    frames_with_opposing_overlapped_faces, _ = payload
    def face_to_objects(face):
        from esper.stdlib import face_landmarks_to_dict
        return [{
            'type': 'bbox',
            'bbox_x1': face.x[0], 'bbox_x2': face.x[1],
            'bbox_y1': face.y[0], 'bbox_y2': face.y[1]
        }, face_landmarks_to_dict(face.payload['landmarks'])]
    def update(acc, frame):
        def accumulate_faces(a, face):
            return a+face_to_objects(face)
        return acc + frame.payload.fold(accumulate_faces, [])
    return frames_with_opposing_overlapped_faces.fold(update, [])

answer = kissing(list(range(12)))
print("Query finished. Preparing VGrid.")
display_result(convert_to_1d_collection(answer.map_payload(payload_to_vgrid_objects)), display_payload=True)

# Action Shots

In [ ]:
def action_shots(vids=None):
    from query.models import Shot
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d import IntervalSet3D, Interval3D
    from rekall.temporal_predicates import meets_before, overlaps, equal
    from rekall.interval_set_3d_utils import T,P,XY,or_preds,X,Y
    from django.db.models import ExpressionWrapper, FloatField
    from esper.captions import get_all_segments
    from rekall.merge_ops import payload_first, payload_plus, merge_named_payload
    import numpy as np
    from tqdm import tqdm

    NUM_SHOTS=5
    MAX_SHOT_DURATION=0.8
    BRIGHTNESS_THRESHOLD = 20.0
    MAX_NUM_WORDS_PER_SECOND = 1.0
    
    if vids is None:
        vids = [v.id for v in Video.objects.filter(ignore_film=False)]
    
    shots_qs = Shot.objects.annotate(
        duration = ExpressionWrapper((F('max_frame') - F('min_frame')) / F('video__fps'), output_field=FloatField())
    ).filter(
        duration__lt=MAX_SHOT_DURATION,
        duration__gt=0.,
        cinematic=True,
        video_id__in=vids,
    )
    total=shots_qs.count()
    
    # Shot_IS<>
    short_shots = DomainIntervalCollection.from_django_qs(shots_qs, progress=True, total=total)
    
    def get_all_frames(short_shots):
        def update(frames, shot):
            return frames + list(range(shot.t[0], shot.t[1]+1))
        return short_shots.fold(update, [])
    
    video_to_frame_numbers = get_all_frames(short_shots)
    all_frames = DomainIntervalCollection({})
    for video, frames in tqdm(video_to_frame_numbers.items()):
        # Getting brightness
        qs = Frame.objects.filter(video_id=video, number__in=frames, regularly_sampled=True).order_by('number')
        all_frames = all_frames.union(DomainIntervalCollection.from_django_qs(
            qs, schema={'t1':'number', 't2': 'number', 'payload': 'brightness'}), profile=False)
    
    def select_second(p):
        return p[1]
    
    # Shot_IS<Frame_IS>
    shots_with_brightness = short_shots.collect_by_interval(
        all_frames, T(overlaps()), time_window=0, filter_empty=False
    ).map_payload(select_second)
    
    # Sequence_IS<Shot_IS<Frame_IS>>
    one_shots = shots_with_brightness.collect_by_interval(
        shots_with_brightness, T(equal()), time_window=0).map_payload(select_second)
    n_shots = one_shots
    for n in range(2, NUM_SHOTS+1):
        print("Constructing {} consecutive short shots".format(n))
        n_shots = n_shots.merge(
            one_shots,
            T(meets_before(epsilon=1)),
            payload_merge_op = IntervalSet3D.union,            
            time_window=1, profile=False)
        print('There are {} videos with {} consecutive short shots'.format(
                 len(n_shots.get_grouped_intervals()), n))
        
    def merge_shots(seq1, seq2):
        return seq1.union(seq2.minus(seq1))
    coalesced_n_shots = n_shots.temporal_coalesce(payload_merge_op=merge_shots)
    
    def bright_enough(shots):
        # Check if any shots is above mean brightness threshold
        def compute_avg_brightness(frames):
            ret = frames.fold(lambda acc, f: acc+f.payload, 0)
            if not frames.empty():
                ret = ret / frames.size()
            return ret
        return shots.map_payload(compute_avg_brightness).fold(
            lambda acc, shot: acc or shot.payload > BRIGHTNESS_THRESHOLD)
            
    n_bright_shots = n_shots.filter(P(bright_enough))
    
    vids = n_bright_shots.get_grouped_intervals().keys()
    results = get_all_segments(vids)
    fps_map = dict((i, Video.objects.get(id=i).fps) for i in vids)
    
    # Word_IS<fps>
    caption_results = DomainIntervalCollection({
        video_id: IntervalSet3D([Interval3D(
            (word[0] * fps_map[video_id], word[1] * fps_map[video_id]),
            payload = fps_map[video_id])
            for word in words])
        for video_id, words in results
    })
    
    def has_few_words(seq):
        _, words = seq.payload
        n_words = words.size()
        if n_words == 0:
            return True
        time = seq.length() / words.get_intervals()[0].payload
        return n_words / time <= MAX_NUM_WORDS_PER_SECOND
    
    # Seq_IS<(Shot_IS<Frame_IS>, Word_IS)>
    n_bright_shots_with_few_words = n_bright_shots.collect_by_interval(
        caption_results,
        T(overlaps()),
        time_window=0,
        filter_empty=False).filter(has_few_words)
    
    # Seq_IS<Shot_IS<Frame_IS>>
    action_shots = coalesced_n_shots.filter_against(
        n_bright_shots_with_few_words,
        T(overlaps()),
        time_window=0)
    
    return action_shots

answer = action_shots()
display_result(convert_to_1d_collection(answer))

# Conversations with Identity Labels

In [ ]:
def conversationsq(vids, progress=True):
    from query.models import FaceCharacterActor, Shot
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d import IntervalSet3D, Interval3D
    from rekall.temporal_predicates import before, overlaps, equal
    from rekall.interval_set_3d_utils import T,P,XY,or_preds,X,Y
    from django.db.models import F
    
    faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
        min_frame=F('face__frame__number'),
        max_frame=F('face__frame__number'),
        video_id=F('face__frame__video_id'),
        bbox_x1=F('face__bbox_x1'),
        bbox_y1=F('face__bbox_y1'),
        bbox_x2=F('face__bbox_x2'),
        bbox_y2=F('face__bbox_y2'),
        character_name=F('characteractor__character__name')
    ).filter(face__frame__video_id__in=vids)
    
    total = faces_with_character_actor_qs.count()
    
    schema = DomainIntervalCollection.django_bbox_default_schema()
    schema['payload'] = 'character_name'    
    all_faces = DomainIntervalCollection.from_django_qs(faces_with_character_actor_qs, schema, progress=progress,
                                                    total=total)
    
    vids = all_faces.get_grouped_intervals().keys()
    if len(vids) == 0:
        return DomainIntervalCollection({})
    
    shots_qs = Shot.objects.filter(
        video_id__in = vids,
        cinematic = True,
    )
    total = shots_qs.count()
    shots = DomainIntervalCollection.from_django_qs(
        shots_qs,
        progress=progress, total=total
    )
    
    def select_second(p):
        return p[1]
    
    # Shot_IS<Face_IS>
    shots_with_frames = shots.collect_by_interval(
        all_faces, T(overlaps()), time_window=0, filter_empty=True).map_payload(select_second)
    
    def group_characters(faces):
        def get_char(face):
            return face.payload
        def merge(char, faces):
            merged_interval = faces.fold(Interval3D.merge)
            merged_interval.payload = (char, faces)
            return merged_interval
        return faces.group_by(get_char, merge)
    
    # Shot_IS<Char_IS<(char, Face_IS)>>
    shots_with_chars = shots_with_frames.map_payload(group_characters)
    
    def cross_product_chars(chars1, chars2):
        def get_chars(chars):
            def update(acc, char):
                acc.append(char.payload[0])
                return acc
            return chars.fold(update, [])
        chars_in_1 = get_chars(chars1)
        chars_in_2 = get_chars(chars2)
        result = []
        for charA in chars_in_1:
            for charB in chars_in_2:
                if charA != charB:
                    result.append((charA, charB))
        return result
            
    # Seq_IS<[(char, char)]>
    two_shots = shots_with_chars.merge(
        shots_with_chars,
        T(before(max_dist=1)),
        payload_merge_op=cross_product_chars,
        time_window=1
    )
    
    def sequences_share_face_pair(list1, list2):
        for A1, B1 in list1:
            for A2, B2 in list2:
                if ((A1==A2 and B1==B2) or
                    (A1==B2 and B1==A2)):
                    return True
        return False
    def merge_face_pairs(list1, list2):
        return list1+list2
    
    def coalesce(self, predicate, payload_merge_op):
        from rekall.interval_set_3d import IntervalSet3D
        # State is (new, current)
        def update(state, interval):
            new, current = state
            updated_current = []
            size = len(new)+len(current)
            for cur in current:
                # No more intervals will overlap with cur
                if cur.t[1] < interval.t[0]:
                    new.append(cur)
                else:
                    updated_current.append(cur)
            matched = None
            for i, cur in enumerate(updated_current):
                if predicate(cur, interval):
                    matched = i
                    break
            if matched is None:
                updated_current.append(interval)
            else:
                updated_current[matched] = updated_current[matched].merge(interval, payload_merge_op)
            return new, updated_current
        converged = False
        while not converged:
            old = self.size(profile=False)
            self = self.fold_to_set(update, ([],[]), acc_to_set=lambda state: IntervalSet3D(state[0]+state[1]),
                                    profile=False)
            converged = old == self.size(profile=False)
            # print(old, self.size(profile=False))
        return self
    
    conv_candidates = coalesce(two_shots, P(sequences_share_face_pair), merge_face_pairs)
    
    def num_shots_at_least(n):
        def pred(shots):
            return shots.size() >= n
        return pred
    
    # Conv_IS<Shot_IS<Char_IS<(char, Face_IS)>>>
    convs = conv_candidates.collect_by_interval(
        shots_with_chars, T(overlaps()), time_window=0, filter_empty=True
    ).map_payload(select_second
    ).filter(P(num_shots_at_least(3)))
    
    
    return convs

answer = conversationsq([380])
display_result(convert_to_1d_collection(answer))

## Conversations with Embedding

In [ ]:
def conversations_emb(vids, progress=True):
    from query.models import Shot, Face
    import esper.face_embeddings as fe
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.temporal_predicates import overlaps, before
    from rekall.interval_set_3d_utils import T,P
    from django.db.models import F
    import numpy as np
    
    EMBEDDING_EQUALITY_THRESHOLD = 1.
    ONE_FRAME = 1
    MAX_CLUSTER_RETRY = 10
    
    def select_second(p):
        return p[1]
    
    # Use face closest to cluster centroid as center of cluster
    def compute_center(fids):
        mean = fe.mean(fids)
        dists = fe.dist(fids, targets=[mean])
        center_id = min(zip(dists, fids))[1]
        return fe.features([center_id])[0]
    
    # Kmeans on small data can hang
    def run_kmeans(fids, num_cluster, num_attempt=0):
        try:
            return fe.kmeans(fids, num_cluster, max(len(fids),25))
        except RuntimeError:
            if num_attempt > MAX_CLUSTER_RETRY:
                raise RuntimeError("Failed to cluster fids {0} into {1} clusters".format(fids, num_cluster))
            print("Failed to cluster! Retrying")
            return run_kmeans(fids, num_cluster, num_attempt+1)
    
    # Frames<Faces<ID>>
    # Returns a list of (ClusterCenterEmbedding, FaceIDsInCluster)
    def cluster_faces_and_compute_center(frames):
        num_people_in_shot = max(frame.payload.size() for frame in frames.get_intervals())
        all_faces = frames.split(lambda f: f.payload)
        face_ids = [face.payload for face in all_faces.get_intervals()]
        # cluster_assignment is list of (FaceID, ClusterID)
        if num_people_in_shot == 1:
            cluster_assignments = [(fid, 0) for fid in face_ids]
        else:
            cluster_assignments = run_kmeans(face_ids, num_people_in_shot)
        clusters = []
        
        for i in range(num_people_in_shot):
            faces_in_cluster = [val[0] for val in cluster_assignments if val[1]==i]
            center_face_emb = compute_center(faces_in_cluster)
            clusters.append((center_face_emb, faces_in_cluster))
        return clusters
    
    def same_face(face1, face2):
        return np.linalg.norm(face1-face2) < EMBEDDING_EQUALITY_THRESHOLD
    
    def cross_product_faces(clusters1, clusters2):
        pairs = []
        for c1 in clusters1:
            for c2 in clusters2:
                if not same_face(c1[0], c2[0]):
                    pairs.append((c1, c2))
        return pairs
    
    def sequences_share_face_pair(pairs1, pairs2):
        for c1, c2 in pairs1:
            for d1, d2 in pairs2:
                if (same_face(c1[0], d1[0]) and same_face(c2[0], d2[0])) or (
                    same_face(c1[0], d2[0]) and same_face(c2[0], d1[0])):
                    return True
        return False
    
    def merge_face_pairs(pairs1, pairs2):
        return pairs1 + pairs2
    
    def coalesce(self, predicate, payload_merge_op):
        from rekall.interval_set_3d import IntervalSet3D
        # State is (new, current)
        def update(state, interval):
            new, current = state
            updated_current = []
            size = len(new)+len(current)
            for cur in current:
                # No more intervals will overlap with cur
                if cur.t[1] < interval.t[0]:
                    new.append(cur)
                else:
                    updated_current.append(cur)
            matched = None
            for i, cur in enumerate(updated_current):
                if predicate(cur, interval):
                    matched = i
                    break
            if matched is None:
                updated_current.append(interval)
            else:
                updated_current[matched] = updated_current[matched].merge(interval, payload_merge_op)
            return new, updated_current
        converged = False
        while not converged:
            old = self.size(profile=False)
            self = self.fold_to_set(update, ([],[]), acc_to_set=lambda state: IntervalSet3D(state[0]+state[1]),
                                    profile=False)
            converged = old == self.size(profile=False)
        return self
    
    def num_shots_at_least(n):
        def pred(shots):
            return shots.size() >= n
        return pred
    
    faces_qs = Face.objects.annotate(
        min_frame = F('frame__number'),
        max_frame = F('frame__number'),
        video_id = F('frame__video_id'),
    ).filter(
        video_id__in = vids,
        frame__regularly_sampled=True
    )
    total = faces_qs.count()
    # Faces<ID>
    faces = DomainIntervalCollection.from_django_qs(
        faces_qs,
        DomainIntervalCollection.django_bbox_default_schema(),
        progress=progress, total=total)
    
    # Frames<Faces<ID>>
    frames_with_faces = faces.group_by_time(profile=progress)
    
    shots_qs = Shot.objects.filter(
        video_id__in=vids,
        cinematic=True
    )
    total = shots_qs.count()
    # Shots<Frames<Faces<ID>>>
    shots = DomainIntervalCollection.from_django_qs(
        shots_qs,
        progress=progress, total=total
    ).collect_by_interval(
        frames_with_faces,
        T(overlaps()),
        filter_empty=True,
        time_window=0, profile=progress,
    ).map_payload(select_second)
    

    # Shots<FaceClusters>
    shots_with_face_clusters = shots.map_payload(cluster_faces_and_compute_center, profile=progress)
    

    # Sequences<FacePairList>
    two_shots = shots_with_face_clusters.merge(
        shots_with_face_clusters,
        T(before(max_dist=ONE_FRAME)),
        payload_merge_op=cross_product_faces,
        time_window=1,
        profile=progress,
    )
    # Sequences<FacePairList>
    conv_candidates = coalesce(two_shots, P(sequences_share_face_pair), merge_face_pairs)

    conv_with_shots = conv_candidates.collect_by_interval(
        shots_with_face_clusters,
        T(overlaps()),
        filter_empty=True,
        time_window=0, profile=progress,
    ).map_payload(select_second).filter(
        P(num_shots_at_least(3))
    )
    
    return conv_with_shots   

answer = conversations_emb([380], True)
display_result(convert_to_1d_collection(answer))

# ShotScale

In [ ]:
def shot_scale_q(vids, progress=False):
    from query.models import Face, PoseMeta, Shot
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d import IntervalSet3D, Interval3D
    from rekall.temporal_predicates import before, overlaps, equal
    from rekall.interval_set_3d_utils import T,P,XY,or_preds,X,Y
    from esper import pose_wrapper as pw
    from esper import shot_scale
    from django.db.models import F
    
    shots_qs = Shot.objects.filter(video_id__in=vids, cinematic=True)
    total = shots_qs.count()
    # Shot_IS
    shots = DomainIntervalCollection.from_django_qs(shots_qs, progress=progress, total=total)
    
    faces_qs= Face.objects.filter(frame__video_id__in=vids, frame__regularly_sampled=True
                                 ).annotate(video_id=F('frame__video__id'),
                                            number=F('frame__number'))
    poses_qs = PoseMeta.objects.filter(frame__video_id__in=vids, frame__regularly_sampled=True
                                 ).annotate(video_id=F('frame__video__id'),
                                            number=F('frame__number'))
    
    total = faces_qs.count()
    all_faces = DomainIntervalCollection.from_django_qs(faces_qs, schema={
        't1':'number', 't2':'number', 'x1':'bbox_x1', 'x2':'bbox_x2', 'y1':'bbox_y1', 'y2':'bbox_y2'
    }, progress=progress, total=total)
    
    total = poses_qs.count()
    all_poses = DomainIntervalCollection.from_django_qs(poses_qs, schema={
        't1':'number', 't2':'number', 'payload':'id'
    }, progress=progress, total=total)
    
    print("Loading {0} Poses".format(total))
    
    def get_pose_map(poses_qs):
        poses = pw.get(poses_qs)
        print("Poses loaded")
        return {pose.id: pose for pose in poses}
    
    def get_pose(pose_map):
        def map_fn(payload):
            return pose_map[payload]
        return map_fn
    
    all_poses = all_poses.map_payload(get_pose(get_pose_map(poses_qs)), profile=progress)
    
    def add_scale_to_face(face):
        face.payload = shot_scale.face_height_to_shot_scale(face.height())
        return face
    
    # Face_IS<Scale>
    all_faces_with_scale = all_faces.map(add_scale_to_face, profile=progress)
    
    def add_scale_to_pose(pose):
        return {
            'pose': pose,
            'scale': shot_scale.pose_keypoints_to_shot_scale(pose.pose_keypoints())
        }
    # Pose_IS<{'pose','scale'}>
    all_poses_with_scale = all_poses.map_payload(add_scale_to_pose, profile=progress)
    
    # Frame_IS
    all_frames_with_signal = all_faces.union(all_poses).group_by_time(profile=progress)
    
    def select_second(p):
        return p[1]
    
    # Frame_IS<(Face_IS, Pose_IS)>
    frames_with_faces_and_poses = all_frames_with_signal.collect_by_interval(
        all_faces_with_scale,
        T(overlaps()),
        filter_empty=False,
        time_window=0, profile=progress,
    ).map_payload(select_second).collect_by_interval(
        all_poses_with_scale,
        T(overlaps()),
        filter_empty=False,
        time_window=0, profile=progress
    )
    
    def add_scale_to_frame(payload):
        faces, poses = payload
        def take_max_face(acc, face):
            return max(acc, face.payload)
        def take_max_pose(acc, pose):
            return max(acc, pose.payload['scale'])
        max_face_scale = faces.fold(take_max_face, shot_scale.ShotScale.UNKNOWN)
        max_pose_scale = poses.fold(take_max_pose, shot_scale.ShotScale.UNKNOWN)
        return (max(max_face_scale, max_pose_scale), faces, poses)
    
    # Frame_IS<(Scale, Face_IS, Pose_IS)>
    frames_with_scale = frames_with_faces_and_poses.map_payload(add_scale_to_frame, profile=progress)
    
    # Shot_IS<Frame_IS<(Scale, Face_IS, Pose_IS)>>
    shots_with_frames = shots.collect_by_interval(
        frames_with_scale,
        T(overlaps()),
        filter_empty=False,
        time_window=0, profile=progress,
    ).map_payload(select_second)
    
    def get_mode(scales):
        count = {}
        for s in shot_scale.ShotScale:
            count[s] = 0
        for s in scales:
            count[s]+=1
        best_count = 1
        best_scale=shot_scale.ShotScale.UNKNOWN
        for s in shot_scale.ShotScale:
            if count[s]>=best_count:
                best_count = count[s]
                best_scale = s
        return best_scale
    
    def add_scale_to_shot(frames):
        def update(acc, frame):
            acc.append(frame.payload[0])
            return acc
        scales = frames.fold(update, [])
        mode = get_mode(scales)
        return mode, frames
    
    # Shot_IS<(Scale, Frame_IS<(Scale, Face_IS<Scale>, Pose_IS<(Scale, PoseKeypoints)>)>)>
    shots_with_scale = shots_with_frames.map_payload(add_scale_to_shot, profile=progress)
    return shots_with_scale

def payload_to_vgrid_objects(payload):
    from rekall.interval_set_3d_utils import P
    # Frame_IS<(Scale, Face_IS, Pose_IS)>
    scale, frames = payload
    def face_to_object(face):
        return {
            'type': 'bbox',
            'bbox_x1': face.x[0], 'bbox_x2': face.x[1],
            'bbox_y1': face.y[0], 'bbox_y2': face.y[1]
        }
    
    def pose_to_object(pose):
        from esper.stdlib import pose_to_dict
        return pose_to_dict(pose.payload['pose'])
    
    def face_objects_at_scale(faces, scale):
        faces = faces.filter(P(lambda p:p==scale))
        def update(acc, face):
            acc.append(face_to_object(face))
            return acc
        return faces.fold(update, [])
    
    def pose_objects_at_scale(poses, scale):
        poses = poses.filter(P(lambda p:p['scale']==scale))
        def update(acc, pose):
            acc.append(pose_to_object(pose))
            return acc
        return poses.fold(update, [])
    
    frames = frames.filter(P(lambda p:p[0]==scale))
    def update(acc, frame):
        _, faces, poses = frame.payload
        return acc + face_objects_at_scale(faces, scale) + pose_objects_at_scale(poses, scale)
    return frames.fold(update, [])

answer = shot_scale_q([553], True)
print("Query Done. Preparing for VGrid.")
display_result(convert_to_1d_collection(answer.filter(
    lambda i:i.payload[0]==7).map_payload(payload_to_vgrid_objects)), display_payload=True)

## ShotScale from frame-level scale

In [ ]:
def shot_scale_from_frame(vids, progress=True):
    from query.models import Frame, Shot
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d import IntervalSet3D, Interval3D
    from rekall.temporal_predicates import before, overlaps, equal
    from rekall.interval_set_3d_utils import T,P,XY,or_preds,X,Y
    from esper import shot_scale
    from django.db.models import F
    
    def shot_scale_lookup_table():
        from query.models import ShotScale
        return {s.id: shot_scale.ShotScale[s.name.upper()] for s in ShotScale.objects.all()}
    
    SCALE_LOOKUP = shot_scale_lookup_table()
    
    shots_qs = Shot.objects.filter(video_id__in=vids, cinematic=True)
    total = shots_qs.count()
    # Shot_IS
    shots = DomainIntervalCollection.from_django_qs(shots_qs, progress=progress, total=total)
    
    frames_qs = Frame.objects.filter(video_id__in=vids, regularly_sampled=True).annotate(
        min_frame = F('number'),
        max_frame = F('number'),
    )
    total = frames_qs.count()
    frames = DomainIntervalCollection.from_django_qs(
        frames_qs,
        with_payload=lambda f: SCALE_LOOKUP[f.shot_scale_id],
        progress=progress, total=total)
    
    # Shot_IS<Frame_IS<Scale>>
    shots_with_frames = shots.collect_by_interval(
        frames,
        T(overlaps()),
        filter_empty=False,
        time_window=0,
    ).map_payload(lambda p:p[1])
    
    def get_mode(scales):
        count = {}
        for s in shot_scale.ShotScale:
            count[s] = 0
        for s in scales:
            count[s]+=1
        best_count = 1
        best_scale=shot_scale.ShotScale.UNKNOWN
        for s in shot_scale.ShotScale:
            if count[s]>=best_count:
                best_count = count[s]
                best_scale = s
        return best_scale
    
    # Frames -> (Scale, Frames)
    def compute_scale(frames):
        scales = [f.payload for f in frames.get_intervals()]
        mode = get_mode(scales)
        return mode, frames
    
    shots_with_scale= shots_with_frames.map_payload(compute_scale)
    return shots_with_scale
    
answer = shot_scale_from_frame([380])
print("Query Done. Preparing for VGrid.")
display_result(convert_to_1d_collection(answer.filter(lambda s:s.payload[0]==7)))

# Multiprocess

In [ ]:
def dummy_func(vids):
    from rekall.interval_set_3d import IntervalSet3D, Interval3D
    from rekall.domain_interval_collection import DomainIntervalCollection
    from query.models import Video
        
    vs = Video.objects.filter(id__in=vids)
    return DomainIntervalCollection({
        v.id: IntervalSet3D([Interval3D((0, v.num_frames-1))]) for v in vs})

def run_dummy():
    from rekall.runtime import Runtime
    from query.models import Video
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = Runtime.inline()
    return rt.run(dummy_func, vids, profile=True, progress=True)

def run_dummy_gen():
    from rekall.runtime import Runtime
    from query.models import Video
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = Runtime.inline()
    return rt.get_result_iterator(dummy_func, vids)

# answer, err = run_dummy()
display_result(convert_to_1d_collection(topN(run_dummy_gen(), n=10)))

## Hermione in the middle

In [ ]:
def run_hp():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video
    import ipyparallel as ipp
    
    c = ipp.Client(profile='local')
    
    vids = [v.id for v in Video.objects.filter(name__contains="harry potter")]
    rt = get_runtime_for_ipython_cluster(c)
    return rt.run(hermione_in_the_middle, vids, profile=False, progress=True)[0]

def run_hp_gen():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video
    import ipyparallel as ipp
    
    c = ipp.Client(profile='local')
    
    vids = [v.id for v in Video.objects.filter(name__contains='harry')]
    rt = get_runtime_for_ipython_cluster(c)
    return rt.get_result_iterator(hermione_in_the_middle, vids)

#answer = run_hp()
display_result(convert_to_1d_collection(topN(run_hp_gen())))

## Kissing

In [ ]:
def run_kissing():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    import ipyparallel as ipp
    from query.models import Video
    
    c = ipp.Client(profile='local')
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(c)
    return rt.run(kissing, vids, profile=False, progress=True, chunksize=5)[0]

def run_kissing_gen():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    import ipyparallel as ipp
    from query.models import Video
    
    c = ipp.Client(profile='local')
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(c)
    return rt.get_result_iterator(kissing, vids, chunksize=5)

display_result(convert_to_1d_collection(topN(run_kissing_gen(), n=25)))

## Action Shots

In [ ]:
def run_action_shots():
    import ipyparallel as ipp
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video
    
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile="local"))
    return rt.run(action_shots, vids, profile=False, progress=True, chunksize=2)[0]

def run_action_shots_gen():
    import ipyparallel as ipp
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video
    
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile="local"))
    return rt.get_result_iterator(action_shots, vids, chunksize=2)

display_result(convert_to_1d_collection(topN(run_action_shots_gen(), n=1000)))

## Conversations with Identity

In [ ]:
def run_conversations():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video, FaceCharacterActor
    import ipyparallel as ipp
    from django.db.models import F
    vids = [v.vid for v in 
            FaceCharacterActor.objects.annotate(
                vid=F('face__frame__video_id')).distinct('face__frame__video_id')
           ]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile="local"))
    return rt.run(conversationsq, vids, profile=False, progress=True, chunksize=10)[0]

def run_conversations_gen():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video, FaceCharacterActor
    import ipyparallel as ipp
    from django.db.models import F
    
    vids = [v.vid for v in 
            FaceCharacterActor.objects.annotate(
                vid=F('face__frame__video_id')).distinct('face__frame__video_id')
           ]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile="local"))
    return rt.get_result_iterator(conversationsq, vids, chunksize=1)

display_result(convert_to_1d_collection(topN(run_conversations_gen(), n=1000)))

### Conversations with Embeddings

In [ ]:
def run_conversations_emb():
    from esper.rekall_parallel import get_worker_pool_factory_for_ipython_cluster
    from query.models import Video
    from rekall.runtime import Runtime
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = Runtime(get_worker_pool_factory_for_ipython_cluster(ipp.Client(profile="local")))
    return rt.run(conversations_emb, vids, progress=True, chunksize=3)[0]

def run_conversations_emb_gen():
    from esper.rekall_parallel import get_worker_pool_factory_for_ipython_cluster
    from query.models import Video
    from rekall.runtime import Runtime
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = Runtime(get_worker_pool_factory_for_ipython_cluster(ipp.Client(profile="local")))
    return rt.get_result_iterator(conversations_emb, vids)

display_result(convert_to_1d_collection(topN(run_conversations_emb_gen())))

## ShotScale

In [ ]:
def shot_scale_no_payload(vids):
    return shot_scale_q(vids, progress=True).map_payload(lambda p:p[0])

def payload_to_vgrid_objects(payload):
    from rekall.interval_set_3d_utils import P
    # Frame_IS<(Scale, Face_IS, Pose_IS)>
    scale, frames = payload
    def face_to_object(face):
        return {
            'type': 'bbox',
            'bbox_x1': face.x[0], 'bbox_x2': face.x[1],
            'bbox_y1': face.y[0], 'bbox_y2': face.y[1]
        }

    def pose_to_object(pose):
        from esper.stdlib import pose_to_dict
        return pose_to_dict(pose.payload['pose'])

    def face_objects_at_scale(faces, scale):
        faces = faces.filter(P(lambda p:p==scale))
        def update(acc, face):
            acc.append(face_to_object(face))
            return acc
        return faces.fold(update, [])

    def pose_objects_at_scale(poses, scale):
        poses = poses.filter(P(lambda p:p['scale']==scale))
        def update(acc, pose):
            acc.append(pose_to_object(pose))
            return acc
        return poses.fold(update, [])

    frames = frames.filter(P(lambda p:p[0]==scale))
    def update(acc, frame):
        _, faces, poses = frame.payload
        return acc + face_objects_at_scale(faces, scale) + pose_objects_at_scale(poses, scale)
    return frames.fold(update, [])

def shot_scale_display_payload(scale_to_find):
    def query(vids):
        from rekall.interval_set_3d_utils import P
        return shot_scale_q(
            vids, progress=False
        ).filter(
            P(lambda p: p[0]==scale_to_find)
        )
    return query

def run_shot_scale():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from query.models import Video
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile='local'))
    return rt.run(shot_scale_q, vids[::10], progress=True, chunksize=1)[0]

def run_shot_scale_gen():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from rekall.runtime import Runtime
    from query.models import Video
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile='local'))
    return rt.get_result_iterator(shot_scale_display_payload(6), vids)

answer = topN(run_shot_scale_gen(), n=25)
display_result(convert_to_1d_collection(answer.map_payload(payload_to_vgrid_objects)), display_payload=True)

### ShotScale from frame-level scale

In [ ]:
def run_shot_scale_from_frame():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from rekall.runtime import disjoint_domain_combiner
    from query.models import Video
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile='local'))
    return rt.run(shot_scale_from_frame, vids, progress=True, chunksize=2, profile=True, combiner=disjoint_domain_combiner)[0]

def shot_scale_from_frames_display_payload(scale_to_find):
    def query(vids):
        from rekall.interval_set_3d_utils import P
        return shot_scale_from_frame(
            vids, progress=False
        ).filter(
            P(lambda p: p[0]==scale_to_find)
        )
    return query

def run_shot_scale_from_frame_gen():
    from esper.rekall_parallel import get_runtime_for_ipython_cluster
    from rekall.runtime import disjoint_domain_combiner
    from query.models import Video
    import ipyparallel as ipp
    
    vids = [v.id for v in Video.objects.all()]
    rt = get_runtime_for_ipython_cluster(ipp.Client(profile='local'))
    return rt.get_result_iterator(shot_scale_from_frames_display_payload(1), vids)


answer = topN(run_shot_scale_from_frame_gen(), n=25)
display_result(convert_to_1d_collection(answer))

# Scratchpad

In [ ]:
answer.filter(lambda i:i.t[0]==120401).get_allintervals()[380].get_intervals()[0].payload.map_payload(lambda char: char.fold(lambda acc, c: acc+[c.payload[0]], []))

In [ ]:
answer.get_allintervals()[32].get_intervals()[0].payload[0].get_intervals()[0].payload.get_intervals()

In [ ]:
def payload_to_vgrid_objects(payload):
    from rekall.interval_set_3d_utils import P
    # Frame_IS<(Scale, Face_IS, Pose_IS)>
    scale, frames = payload
    def face_to_object(face):
        return {
            'type': 'bbox',
            'bbox_x1': face.x[0], 'bbox_x2': face.x[1],
            'bbox_y1': face.y[0], 'bbox_y2': face.y[1]
        }
    
    def pose_to_object(pose):
        from esper.stdlib import pose_to_dict
        return pose_to_dict(pose.payload['pose'])
    
    def face_objects_at_scale(faces, scale):
        faces = faces.filter(P(lambda p:p==scale))
        def update(acc, face):
            acc.append(face_to_object(face))
            return acc
        return faces.fold(update, [])
    
    def pose_objects_at_scale(poses, scale):
        poses = poses.filter(P(lambda p:p['scale']==scale))
        def update(acc, pose):
            acc.append(pose_to_object(pose))
            return acc
        return poses.fold(update, [])
    
    frames = frames.filter(P(lambda p:p[0]==scale))
    def update(acc, frame):
        _, faces, poses = frame.payload
        return acc + face_objects_at_scale(faces, scale) + pose_objects_at_scale(poses, scale)
    return frames.fold(update, [])

display_result(convert_to_1d_collection(answer.map_payload(payload_to_vgrid_objects)), display_payload=True)

In [ ]:
qs=Face.objects.all()[:3]
[q.id for q in qs]

In [ ]:
import esper.face_landmarks_wrapper as flw

In [ ]:
flw.get_from_face_ids([3,5,6])

In [ ]:
from django.db.models import Count
from query.models import Face
Face.objects.values("frame__video_id").annotate(num_faces=Count('id')).order_by('num_faces')

In [ ]:
from query.models import Frame, Video
v = Video.objects.get(id=1)
Frame.objects.filter(video=v, number=v.num_frames-1)

In [ ]:
from esper.shot_scale import ShotScale as ShotScaleEnum
from rekall.interval_set_3d_utils import P
display_result(convert_to_1d_collection(answer.filter(P(lambda p: p==ShotScaleEnum.EXTREME_LONG))))

In [ ]:
display_result(convert_to_1d_collection(answer.map_payload(payload_to_vgrid_objects, profile=True, parallel=True)), display_payload=True)

In [ ]:
import ipyparallel as ipp
c=ipp.Client(profile='local')

In [ ]:
for h in c.history:
    a = c.get_result(h)
    print(h,a.stdout)

In [ ]:
from rekall.interval_set_3d_utils import P
a = answer.add_domain_to_payload().filter(P(lambda p: p[1]==640))
display_result(convert_to_1d_collection(a))

In [ ]:
"{0:03d}".format(1)

In [ ]:
Shot.objects.filter(cinematic=True).count()

In [ ]:
kissing_answer.get_flattened_intervalset().size()

In [ ]:
shot_answer

In [ ]:
from esper.shot_scale import ShotScale as ShotScaleEnum
frames.filter(lambda f: f.payload==ShotScaleEnum.CLOSE_UP).get_grouped_intervals()[380].get_intervals()[1]

In [ ]:
def save_collection(c, name):
    import pickle
    pickle.dump(c, open('/app/data/{0}'.format(name),'wb'))

In [ ]:
save_collection(shot_answer, 'shot_scales/all_shot_scales.pkl')

In [ ]:
import sys
a = 1
print(sys.getsizeof(answer))

In [ ]:
answer

In [ ]:
save_collection(answer, 'tmp.pkl')

In [ ]:
def foo():
    ret = []
    for i in tqdm(range(1000000)):
        ret.append(len(fe.exists(list(range(1)))))
    print(len(ret))
foo()

In [ ]:
import gc
gc.collect()

In [ ]:
import esper.face_embeddings as fe

In [ ]:
del ret

In [ ]:
200*1024*1024/100000

In [ ]:
pickle

In [ ]:
import ipyparallel as ipp
c =ipp.Client(profile='local')
c.abort()

In [ ]:
from esper.rekall_parallel import get_runtime_for_ipython_cluster
from rekall.runtime import disjoint_domain_combiner
from query.models import Video
import ipyparallel as ipp

vids = [v.id for v in Video.objects.all()]
rt = get_runtime_for_ipython_cluster(c)
answer=rt.run(shot_scale_from_frame, vids, progress=True, chunksize=2, profile=True, combiner=disjoint_domain_combiner)[0]

In [ ]:
a=c[0].pull('shot_scale_from_frame')

In [ ]:
def f(x):
    if x == 25:
        raise RuntimeError("25 not supported")
    return x+1

In [ ]:
amr=c[:].map(f, range(100))

In [ ]:
amr.get()

In [ ]:
%debug

In [ ]:
import sys
sys.version

In [ ]:
from rekall.interval_set_3d_utils import P
vid=345
ints = answer.get_grouped_intervals()[vid]

In [ ]:
vid=619
ints = answer.get_grouped_intervals()[vid]

In [ ]:
ints

In [ ]:
vids_with_action = answer.get_grouped_intervals().keys()
vids_with_action

In [ ]:
a=pickle.load(open('/app/data/conversations/intensification/all_intensification.pkl','rb'))
vids_with_intensification = a.keys()
len(vids_with_intensification)

In [ ]:
a=pickle.load(open('/app/data/conversations/start_wide/all_start_wide.pkl','rb'))
vids_with_start_wide = a.keys()
vids_with_start_wide

In [ ]:
vs = set(vids_with_action) & set(vids_with_intensification) & set(vids_with_start_wide)

In [ ]:
[v.path for v in Video.objects.filter(id__in=vs)]

In [ ]:
[f.vid for f in FaceCharacterActor.objects.annotate(vid=F('face__frame__video_id')).distinct('face__frame__video_id')]